# Train NN v0.7: Adding Classifier (WORK IN PROGRESS)

- generates N training samples with varied energies $K_{\alpha_1}$, energy splitting and ratios
- trains one MLP regressor for each parameter
- Also implements possibility to use classification on fixed energy grid
- The trained NNs are tested on training set, dev set and test set
- No optimization have been made.

Changes:

- Removed:

- Changes:
    - Changed some function parameters: Now X and Y are treated seperately 
- New:
    - global variable ob total number labels saved in a library
    - split_library(library): splits library into training, dev, and test set with 80%,10%,10% ratio
- Bug fixes


#### Import basic libraries 

In [ ]:
"""
Import functions and tools needed
"""
import os
import plotly.offline as py
import plotly.tools as tls
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import pandas as pd
import matplotlib
import math
import scipy.stats as sc
from scipy import interpolate
from scipy.special import wofz
from astropy.modeling.functional_models import Voigt1D 
from scipy.interpolate import UnivariateSpline
# several sklearn functions
from sklearn.neural_network import MLPRegressor as MLPR
from sklearn.neural_network import MLPClassifier as MLPC
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn as skl
import time
from sklearn.externals import joblib
# Redirect stdout
import io 
import sys
# Use subpreocesses
import subprocess
# Use tgz to zip large files into compressed containers
import tarfile
# Use garbage collect to clear unused variables and free memory
import gc
# Export data as different file formats
import msgpack
import h5py
import dill # Saves everything! 
# Extra functions for dictionaries
from itertools import chain
from collections import defaultdict
# Control figure size
matplotlib.rcParams['figure.figsize']=(7,5)
py.init_notebook_mode()
# Use plotly as gifure output
def plotly_show():
    fig = plt.gcf()
    plotlyfig = tls.mpl_to_plotly(fig,resize=True)
    plotlyfig['layout']['showlegend']=True
    py.iplot(plotlyfig)
# define global variables
legend = ['states', 'energies', 'energies_binned', 'splittings', 'ratios', 'features']

#### Define functions

In [ ]:
def calc_area(x, v):
    """
    Calculates the area beneath a spectrum
    """
    width = (x[len(x)-1]-x[0])/(len(x)-1)
    area = 0
    for i in range(len(x)):
        area += v[i] * width
    return area

In [ ]:
def build_library(N, n, x, noise, Set, verbosity):
    """
    Function wrapped around gen_set() for building the training library. 
        - If N is large, generates multiple msgpack files and then zips them.
    """
    start = time.time()
    label_counter = 1
    if ( N > 1e5 ):
        # Create files with size of 1e5 until all spectra are created
        # Check if file already exists
        if (os.path.isfile("./data/"+Set+".tar.bz2") == True):
            sys.stderr.write('Error: File already exists! \n')
            return 0
        tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "x:bz2")
        tar = tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "w:bz2")
        while ( N >= 1e5 ):
            gen_set(int(1e5), n, x, noise, "temp_"+str(label_counter)+".sublibrary", verbosity)
            # Zip files to one compressed container and delete single them afterwards
            tar.add("./data/temp_"+str(label_counter)+".sublibrary.")
            # Remove temporary file
            os.remove("./data/temp_"+str(label_counter)+".sublibrary.")
            N -= 1e5
            label_counter += 1
        tar.close()
        end = time.time()
        if (verbosity > 0):
            print("+++++++++++++")
            print("Time for generating library: %3.2fs" % (end-start))
            print("+++++++++++++")
    else:
        if (os.path.isfile("./data/"+Set+".tar.bz2") == True):
            sys.stderr.write('Error: File already exists! \n')
            return 0
        tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "x:bz2")
        tar = tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "w:bz2")
        gen_set(int(N), n, x, noise, "temp_"+str(label_counter)+".sublibrary", verbosity)
        tar.add("./data/temp_"+str(label_counter)+".sublibrary.")
        os.remove("./data/temp_"+str(label_counter)+".sublibrary.")
        end = time.time()
        if (verbosity > 0):
            print("+++++++++++++")
            print("Time for generating library: %3.2fs" % (end-start))
            print("+++++++++++++")
    tar.close()

In [ ]:
def gen_set(N, n, x, noise, Set, verbosity): 
    """
    - Generates a set with N spectra by using the superposition of TWO Voigt profiles with randomly choosen
        parameters 
            gamma1: HWHM of Lorentzian part of Voigt profile 1 
            gamma2: HWHM of Lorentzian part of Voigt profile 2
            sigma1: Standard uncertainty of Gaussian part of Voigt profile 1
            sigma2: Standard uncertainty of Gaussian part of Voigt profile 2
            epsilons: offset to energy E, dE, Ratios
        The Energy E (K alpha1) is centered around 2014eV 
        Splitting is set to 0.85eV +- 0.05
        Ratios are set to 1.7 pm 0.5
    """
    if (verbosity > 0):
        start = time.time()
    # Definition of some parameters
    gamma1, sigma1 = 0.345, 0.07
    gamma2, sigma2 = 0.36, 0.08
    
    # Creating the empty data dictionary to store data
    k = 5
    X = {
        'states': np.zeros(N),
        'energies': np.zeros(N*k).reshape(N,k),
        'energies_binned': np.zeros(N*k).reshape(N,k),
        'splittings': np.zeros(N*k).reshape(N,k),
        'ratios': np.zeros(N*k).reshape(N,k),
        'features': np.zeros(N*len(x)).reshape(N,len(x))
    }
    
    runtime = np.array(0)
    runtime = np.delete(runtime, 0)
    """
    For loop loops N times to create N spectra. The single spectrum is evaluate and fitted
    on range x to get equal x values as features (Note: When trained on grid defined by x then
    real data must also be sampled on same grid!). File format:
        File dimensions: N x (2 + d), where d is number of grid points resulting from grid x
        [E dE x1 x2 ... xd]
    """
    if (verbosity > 0):
        loop_time_start = time.time()
    for i in range(N):
        # Create empty feature array
        feature_array = np.zeros(len(x))
        ## Generate superposition of n different oxidation station 
        # Save number of states in dictionary
        X["states"][i] = n
        for j in range(n):
            # Generate random distribution (+- 1) around central value of energie E
            E_epsilon = (np.random.random_sample()-0.5)*2
            # Generate random distribution (+- 0.1) around central value of energie dE
            dE_epsilon = (np.random.random_sample()-0.5)*(0.4)
            # Generate random distribution (+- 0.05) around central value of amplitude L1
            dL1 = (np.random.random_sample()-0.5)/15
            # Generate random distribution (+- 0.05) around central value of amplitude L2
            dL2 = (np.random.random_sample()-0.5)/15
            L1 = 0.18 + dL1
            L2 = 0.3 + dL2
            E = 2014 + E_epsilon
            dE = 0.85 + dE_epsilon
            v1 = Voigt1D(x_0=E-dE, amplitude_L=L1, fwhm_L=2*gamma1, fwhm_G=2*sigma1*np.sqrt(2*np.log(2)))
            v2 = Voigt1D(x_0=E, amplitude_L=L2, fwhm_L=2*gamma2, fwhm_G=2*sigma2*np.sqrt(2*np.log(2)))
            # Superpose the states
            feature_array += v1(x)+v2(x)
            # Calculate the ratio of the areas
            R = calc_area(x, v2(x))/calc_area(x,v1(x))
            ### Discretize Energies for classification option
            # Define energy bins
            bins = np.arange(2013,2015+0.01,0.01)
            # Apply this grid on enegery and center energy to the bin center to increase precision
            binned = np.digitize(E, bins, right=True)
            E_binned = bins[binned]#-0.005
            # Save values in dictionary
            X["energies"][i,j+0] = E
            X["energies_binned"][i,j] = E_binned
            X["splittings"][i,j] = dE
            X["ratios"][i,j] = R
        ## Apply noise to data    
        if (noise == True):
            # Normalize spectrum to 1
            amp = np.amax(feature_array)
            feature_array  /= amp
            # Apply poisson noise to the data, magnify amplitudes to get poisson function working
            feature_array =np.multiply(feature_array,0.5*1e3)
            feature_array = np.random.poisson(feature_array)
            # Scale down again
            feature_array = np.divide(feature_array, 0.5*1e4)
            # Fill ddictionary:
            X["features"][i] = feature_array
        else: 
            # Normalize spectrum to 1
            amp = np.amax(feature_array)
            feature_array  /= amp
            # Fill ddictionary:
            X["features"][i] = feature_array
        # Runtime control
        if (verbosity > 0):
            if ( i % (N/10) == 0 ):
                loop_time_end = time.time()
                time_diff = loop_time_end-loop_time_start
                runtime = np.append(runtime, time_diff)
                print("Progress: %i/%i, time for loop: %3.2fs" % (i , N, time_diff))
                loop_time_start = time.time()
    # Save dictinoary as library in .msgpack file
    with open('./data/'+Set,'wb') as f:
        dill.dump(X,f)
    # Some verbosity output
    if (verbosity > 0):
        end = time.time()
        print("Time for generating the "+Set+" set:", end-start)
        if (verbosity > 1):
            plt.figure()
            plt.title("Runtime for generating the data set "+Set)
            plt.plot(runtime, label="Runtime per N/10 loops")
            plt.grid(True)
            plt.legend()
            plotly_show()
    return X

In [ ]:
def load_library(Set, fraction, verbosity):
    """
    Function wrapped around read_set() for loading the training library. 
    """
    start = time.time()
    tar = tarfile.open("./data/"+Set+".tar.bz2", "r:bz2")
    label_counter = 1
    k = 0
    N = 0
    l = 0
    X = {
        'states': np.zeros(N),
        'energies': np.zeros(N*k).reshape(N,k),
        'energies_binned': np.zeros(N*k).reshape(N,k),
        'splittings': np.zeros(N*k).reshape(N,k),
        'ratios': np.zeros(N*k).reshape(N,k),
        'features': np.zeros(N*l).reshape(N,l)
    }
    for i in (tar):
        if ( label_counter > fraction ):
            break
        start_loop = time.time()
        tar.extract(i)
        temp = read_set("temp_"+str(label_counter)+".sublibrary")
        for item in legend:
            X[item] = np.append(X[item], temp[item])
        end_loop = time.time()
        if (verbosity > 0):
            print("Time for unpacking: %3.2fs" % (end_loop-start_loop))
        os.remove("./data/temp_"+str(label_counter)+".sublibrary")
        label_counter += 1
    tar.close()
    end = time.time()
    if (verbosity > 0):
        print("+++++++++++")
        print("Time for loading the library: %3.2fs" % (end-start))
        print("+++++++++++")
    return temp

In [ ]:
def read_set(Set):
    """ 
    Read data and store it in (Nxd) Martix, where N donates 
    the observation (single spectrum) and d the dth feature 
    (datapoint given by choosing x). The data gets fitted 
    by the Splines fit. Also, noise is added when reading the
    data if flag is set.
    """
    start = time.time()
    with open('./data/'+Set,'rb') as f:
        X = dill.load(f)
    end = time.time()
    print("Time for reading "+Set+" set: %3.2fs" % (end-start))
    return X

In [ ]:
def scale_input(X):
    """
    Feature skaling for NN apporach. It is "highly recommended" to scale input data to either [0:1] or [-1:+1] 
    or standardize it to have mean 0 and variance 1
    Source:
    http://scikit-learn.org/stable/modules/neural_networks_supervised.html#regression
    This function standardizes X 
    """
    from sklearn.preprocessing import StandardScaler  
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    N = len(X)
    d = len(X[0]+n_labels)
    scaler.fit(X)  
    X = scaler.transform(X) 
    #for i in range(len(X)):
    #    plt.plot(x,X[i])
    #plt.grid(True)
    #plotly_show()
    return X

In [ ]:
def NN_train(library, model, parameter, scaling, verbosity):
    """
    Trains given model on data X and labels y. Returns trainings score
    """
    start = time.time()
    Xtrain, Ytrain, Xdev, Ydev, Xtest, Ytest = split_library(library)
    X = Xtrain
    y = np.ravel(Ytrain[:,parameter])*1e3 # multiply to create an integer problem
    y = y.astype(int)
    if (scaling == True):
        X = scale_input(X)
    # Set out pipe to catch stdout for getting verbosity output of model.fit
    if (verbosity > 0):
        old_stdout = sys.stdout
        sys.stdout = mystdout = io.StringIO()
    model.fit(X, y)
    # Delete pipe
    if (verbosity > 0):
        sys.stdout = old_stdout
    # Save verbosity output (training loss) in variable
    loss = np.array(0)
    loss = np.delete(loss, 0)
    if (verbosity > 0):
        verbosity_output = mystdout.getvalue()
        verbosity_output = np.array(verbosity_output.split(' '))
        for i in range(4,len(verbosity_output),4):
            if (verbosity_output[i].split('\n')[0] == 'improve'):
                break
            else:
                loss = np.append(loss, float(verbosity_output[i].split('\n')[0]))
    # Save score of training
    score = model.score(X,y)
    end = time.time()
    # Print training statistics depending onb verbosity level
    if (verbosity > 0):
        print("Training time: %3.2f " % (end - start))
        print("Training score: %3.2f " % (score))
        if (verbosity > 1):
            plt.figure()
            plt.title("Training loss per epoch")
            plt.semilogy(loss, label="Loss")
            plt.grid(True)
            plt.legend()
            plotly_show()       
    return score

In [ ]:
def test_envir(X_train, Y_train, X_dev, Y_dev, X_test, Y_test, model, param, verbosity, scaling):
    """
    Trains and tests a NN on a given label
    """
    param = 0
    score = NN_train(X_train, Y_train, model, param, scaling, verbosity = verbosity)
    #Save model via
    joblib.dump(model, './data/1_neural_network.pkl')
    #Load model via
    model2 = joblib.load('./data/1_neural_network.pkl')
    predict_train = model.predict(X_train)
    predict_dev = model.predict(X_dev)
    plt.plot(Y_train[:,param]-predict_train, label=("Train: Loss energy"))
    plt.plot(Y_dev[:,param]-predict_dev, label=("Dev: Loss energy"))
    plt.xlabel("datapoint")
    plt.ylabel("Error in arb. units")
    plt.title("Error on true label")
    abserr_train = np.absolute(Y_train[:,param]-predict_train)
    abserr_train = np.sum(abserr_train)
    abserr_dev = np.absolute(Y_dev[:,param]-predict_dev)
    abserr_dev = np.sum(abserr_dev)
    plt.legend()
    plt.grid(True)
    print("Mean error per prediction in training set in run %3.2f " % (abserr_train/len(X_train)))
    print("Mean error per prediction in dev set in run %3.2f " % (abserr_dev/len(X_dev)))
    plotly_show()    

In [ ]:
def split_library(library, key):
    """
    Function to split a library in training, dev, and test set after good 'ol ratio 80%:10%:10%
    """
    N = len(library["states"])
    N_train = int(N*0.8)
    N_dev = N_test = int(N*0.1)
    Xtrain = library["features"][0:N_train]
    Ytrain = library[key][0:N_train]
    Xdev = library["features"][N_train : N_train + N_dev]
    Ydev = library[key][N_train : N_train + N_dev]
    Xtest = library["features"][N_train + N_dev : N_train + 2*N_dev]
    Ytest = library[key][N_train + N_dev : N_train + 2*N_dev]
    return Xtrain, Ytrain, Xdev, Ydev, Xtest, Ytest

### Begin of testing the algorithms

In [ ]:
"""
Defining (Hyper)Parameters
"""
exp = 3 # Exponent defining the size of the file
factor = 1
N = int(factor*10**(exp)) # Actual value
noisee = True
n = 5 # Number of spectra
comment = "test" # Comment for data file name
data_size = str(int(N/1000))+"k_" # Value for labeling the data (in "kilo samples") 
set_name = data_size+comment+"_library"
x = np.arange(2008,2018,0.05) # Grid for creating and importing data
# Following the definition of the different 
modelR_energy = MLPR(max_iter=2000,  activation="relu", verbose = True)
modelR_splitting = MLPR(max_iter=2000,  activation="relu", verbose = True)
modelR_ratios = MLPR(max_iter=2000,  activation="relu", verbose = True)
modelC_energy = MLPC(max_iter=2000,  activation="relu", verbose = True)
modelC_splitting = MLPC(max_iter=2000,  activation="relu", verbose = True)
modelC_ratios = MLPC(max_iter=2000,  activation="relu", verbose = True)

In [ ]:
build_library(N, n, x, noise = True, Set = "test", verbosity=True)

In [ ]:
library = load_library(Set = "test", fraction = 1000, verbosity=True)